In [1]:
#r "BoSSSpad.dll"
#r "SAIDT.dll"
#r "ApplicationWithIDT.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using ApplicationWithIDT;
using SAIDT;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();


# Run Simulation

In [2]:
string dbPath = @"SAIDT_db" ;
var database = OpenOrCreateDatabase(dbPath);

Creating database 'SAIDT_db'.


In [3]:
Dictionary<string,int> DOFS = new Dictionary<string,int>();


## SinglePhaseField

In [4]:
using SAIDT;
BoSSS.Solution.Application.InitMPI();
BoSSS.Solution.Application.DeleteOldPlotFiles();
var pSAIDT = new SAIDTMain();
var CSAIDT = SAIDTHardCodedControl.StraightShock(
                    dbPath: dbPath,
                    MaxIterations: 100,
                    dgDegree: 0,
                    numOfCellsX: 5,
                    numOfCellsY: 5,
                    OptiNumOfCellsX: 5,
                    OptiNumOfCellsY: 5,
                    agg: 0.4,
                    ImmediatePlotPeriod: -1,
                    optiLevelSetType: OptiLevelSetType.SinglePhaseField,
                    LSDegree: 3
                    );
CSAIDT.SessionName="SAIDT_DGLevelSet";
CSAIDT.ProjectName="SAIDT_DGLevelSet";
pSAIDT.Init(CSAIDT);
pSAIDT.RunSolverMode();
DOFS.Add("SinglePhaseField",pSAIDT.LsTBO.DOFLocal);

rm;
Session ID: f735876d-8434-437b-ad07-7d02f6d6708a, DB path: 'SAIDT_db'
Session directory 'SAIDT_db\sessions\f735876d-8434-437b-ad07-7d02f6d6708a'.
Grid repartitioning method: METIS
Grid repartitioning options: 
Number of cell Weights: 0


*****************************************************************************************
*  X   X   DDD   GGG   -   III  SSS  TTTTT  *
*   X X    D  D G      -    I  S      T     *
*    X     D  D G  GG  -    I   SSS   T     *      XDG Implicit Shock Tracking Solver    
*   X X    D  D G   G  -    I      S  T     *
*  X   X   DDD   GGG   -   III  SSS   T     *
*****************************************************************************************
*  
*  solver:                             SAIDT.SAIDTMain
*  project:                            SAIDT_DGLevelSet
*  session:                            SAIDT_DGLevelSet
*  optimization problem:               FullEnRes
*  merit function:                     L1Merit
*  level set type:                   

Solve_Direct fail in fallback seq (#0): Linear Solver: High residual from direct solver ilPSP.LinSolvers.PARDISO.PARDISOSolver.
    L2 Norm of RHS:         308.17851093006107
    L2 Norm of Solution:    1.8246303514900557
    L2 Norm of Residual:    0.7689723061578801
    Relative Residual norm: 1.0719726790870406E-06
    Matrix Inf norm:        717343.194616476



alpha, M, predM  1, 6.325955990156246, 7.4242382022931075
l2: ||R0||= 9.1579269E-01, f_err=1.997796E+00, f_phi=0E+00, Alpha = 1E+00, Gamma = 1E+00, Kappa = 1E+03, Mu = 1.93E+00, QuadOrder=4
**NOT** terminating now, minimal number of iterations not reached


Starting Opt-Iter No.2: 


Solve_Direct fail in fallback seq (#0): Linear Solver: High residual from direct solver ilPSP.LinSolvers.PARDISO.PARDISOSolver.
    L2 Norm of RHS:         1791.5650682194228
    L2 Norm of Solution:    0.19638120654214247
    L2 Norm of Residual:    0.10810838106799829
    Relative Residual norm: 3.152759673482569E-08
    Matrix Inf norm:        3429007.988692672



alpha, M, predM  1, 13.04199748976565, 2.087552106931953
alpha, M, predM  0.5, 9.845146885020814, 2.0877465249769758
alpha, M, predM  0.25, 5.146831072891832, 2.0878437339994873
alpha, M, predM  0.125, 2.2805293233651867, 2.087892338510743
alpha, M, predM  0.0625, 1.6633879257193844, 2.087916640766371
l2: ||R0||= 8.017086E-01, f_err=1.778283E+00, f_phi=0E+00, Alpha = 6.25E-02, Gamma = 6.67E-01, Kappa = 1E+03, Mu = 4.11E-02, QuadOrder=4
**NOT** terminating now, minimal number of iterations not reached


Starting Opt-Iter No.3: 


...

alpha, M, predM  1, 18.391446288667883, 0.04905490790913772
alpha, M, predM  0.5, 10.069665114243247, 0.049059582737847454
alpha, M, predM  0.25, 6.460038026108451, 0.04906192015220233
alpha, M, predM  0.125, 5.052731555441984, 0.04906308885937976
alpha, M, predM  0.0625, 4.646374923048501, 0.04906367321296848
alpha, M, predM  0.03125, 4.469801957498651, 0.04906396538976284
alpha, M, predM  0.015625, 4.383193857753635, 0.04906411147816002
alpha, M, predM  0.0078125, 4.340386879891293, 0.049064184522358605
alpha, M, predM  0.00390625, 4.319068099220547, 0.0490642210444579
alpha, M, predM  0.001953125, 4.308429753276587, 0.04906423930550755
alpha, M, predM  0.0009765625, 4.303115813301855, 0.049064248436032375
alpha, M, predM  0.00048828125, 4.300460146941874, 0.049064253001294786
alpha, M, predM  0.000244140625, 4.299132639000445, 0.04906425528392599
alpha, M, predM  0.0001220703125, 4.29846896624934, 0.04906425642524159
alpha, M, predM  6.103515625E-05, 4.298137150166996, 0.04906425699

Solve_Direct fail in fallback seq (#0): Linear Solver: High residual from direct solver ilPSP.LinSolvers.PARDISO.PARDISOSolver.
    L2 Norm of RHS:         36.92319203808632
    L2 Norm of Solution:    41.9779788296677
    L2 Norm of Residual:    0.11486208433006158
    Relative Residual norm: 2.5255583518946154E-07
    Matrix Inf norm:        454798.7744725625



alpha, M, predM  1, 18.391713166228914, 0.04905417256548298
alpha, M, predM  0.5, 10.06986125002902, 0.049058847322382935
alpha, M, predM  0.25, 6.460179544731185, 0.049061184700832906
alpha, M, predM  0.125, 5.052799162071659, 0.0490623533900579
alpha, M, predM  0.0625, 4.6464235037711425, 0.04906293773467039
alpha, M, predM  0.03125, 4.469846753573838, 0.04906322990697664
alpha, M, predM  0.015625, 4.383236858644715, 0.04906337599312976
alpha, M, predM  0.0078125, 4.340429112445423, 0.049063449036206325
alpha, M, predM  0.00390625, 4.319109951684399, 0.049063485557744604
alpha, M, predM  0.001953125, 4.308471416802826, 0.049063503818513744
alpha, M, predM  0.0009765625, 4.303157382654035, 0.04906351294889831
alpha, M, predM  0.00048828125, 4.300501669283743, 0.0490635175140906
alpha, M, predM  0.000244140625, 4.299174137856763, 0.049063519796686744
alpha, M, predM  0.0001220703125, 4.298510453367855, 0.049063520937984814
alpha, M, predM  6.103515625E-05, 4.298178631417856, 0.04906352

Solve_Direct fail in fallback seq (#0): Linear Solver: High residual from direct solver ilPSP.LinSolvers.PARDISO.PARDISOSolver.
    L2 Norm of RHS:         36.92307299245753
    L2 Norm of Solution:    3.552678497343413
    L2 Norm of Residual:    0.0037845883884812482
    Relative Residual norm: 8.321459401958085E-09
    Matrix Inf norm:        454798.63635346387



alpha, M, predM  1, 18.391958001030712, 0.04905398877525968
alpha, M, predM  0.5, 10.069904907424295, 0.049058663514204875
alpha, M, predM  0.25, 6.460216586583977, 0.049061000883677465
alpha, M, predM  0.125, 5.05281372012142, 0.04906216956841376
alpha, M, predM  0.0625, 4.646434702096746, 0.049062753910781916
alpha, M, predM  0.03125, 4.46985746916448, 0.04906304608196599
alpha, M, predM  0.015625, 4.383247364764566, 0.04906319216755802
alpha, M, predM  0.0078125, 4.340439547886848, 0.04906326521035404
alpha, M, predM  0.00390625, 4.319120353469322, 0.04906330173175205
alpha, M, predM  0.001953125, 4.308481802186163, 0.04906331999245106
alpha, M, predM  0.0009765625, 4.3031677599443645, 0.04906332912280056
alpha, M, predM  0.00048828125, 4.3005120425547, 0.04906333368797531
alpha, M, predM  0.000244140625, 4.299184509124856, 0.049063335970562685
alpha, M, predM  0.0001220703125, 4.298520823636216, 0.04906333711185638
alpha, M, predM  6.103515625E-05, 4.2981890011867785, 0.04906333768

In [5]:
pSAIDT.ResNorms.SaveToTextFile(pSAIDT.Control.ProjectName+"_ResPlot.txt");
pSAIDT.obj_f_vals.SaveToTextFile(pSAIDT.Control.ProjectName+"_EnResPlot.txt");
pSAIDT.GetResPlot().ToGnuplot().PlotSVG(xRes:600,yRes:500)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -1 
 
 
 
 
 10 0 
 
 
 
 
 10 1 
 
 
 
 
 0 
 
 
 
 
 5 
 
 
 
 
 10 
 
 
 
 
 15 
 
 
 
 
 20 
 
 
 
 
 25 
 
 
 
 
 30 
 
 
 
 
 35 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Iteration 
 
 
 
 
 ||R(z)|| 
 
 
 ||R(z)|| 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M388.5,57.1 L441.9,57.1 M45.6,171.7 L60.0,178.2 L74.4,188.5 L88.8,188.8 L103.3,189.9 L117.7,189.9
 L132.1,202.0 L146.5,228.5 L160.9,239.1 L175.3,241.7 L189.7,244.4 L204.2,257.8 L218.6,271.1 L233.0,322.1
 L247.4,333.1 L261.8,337.5 L276.2,338.9 L290.6,339.6 L305.1,339.8 L319.5,339.9 L333.9,339.9 L348.3,339.9
 L362.7,339.9 L377.1,339.9 L391.5,339.9 L406.0,339.9 L420.4,339.9 L434.8,339.9 L449.2,342.4 L463.6,342.8
 L478.0,342.9 L492.4,343.0 L506.9,343.0 L521.3,343.0 L535.7,343.0 L550.1,343.0 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ||r(z)|| 
 
 
 ||r(z)|| 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M388.5,81.1 L441.9,81.1 M45.6,272.7 L60.0,247.0 L74.4,258.7 L88.8,259.1 L103.3,256.4 L117.7,256.4
 L132.1,320.5 L146.5,312.5 L160.9,323.7 L175.3,325.8 L189.7,328.4 L204.2,340.4 L218.6,350.4 L233.0,411.4
 L247.4,421.5 L261.8,429.2 L276.2,430.6 L290.6,431.3 L305.1,431.5 L319.5,431.6 L333.9,431.6 L348.3,431.6
 L362.7,431.6 L377.1,431.6 L391.5,431.6 L406.0,431.6 L420.4,431.6 L434.8,431.6 L449.2,431.0 L463.6,431.3
 L478.0,431.4 L492.4,431.5 L506.9,431.5 L521.3,431.5 L535.7,431.5 L550.1,431.5 '/>

In [6]:
var si= database.Sessions.Pick(0);
BoSSS.Solution.Application.DeleteOldPlotFiles();
pSAIDT.PlotShadowFields(si);


rm;


## Spline level set

In [7]:
BoSSS.Solution.Application.InitMPI();
var pSAIDT = new SAIDTMain();
var CSAIDT = SAIDTHardCodedControl.StraightShock(
                    dbPath: dbPath,
                    MaxIterations: 100,
                    dgDegree: 0,
                    numOfCellsX: 5,
                    numOfCellsY: 5,
                    OptiNumOfCellsX: 5,
                    OptiNumOfCellsY: 5,
                    agg: 0.4,
                    ImmediatePlotPeriod: -1,
                    optiLevelSetType: OptiLevelSetType.SplineLevelSet,
                    LSDegree: 3,
                    isFarConfig:false
                    );
CSAIDT.SessionName="SAIDT_SplineLevelSet";
CSAIDT.ProjectName="SAIDT_SplineLevelSet";
pSAIDT.Init(CSAIDT);
pSAIDT.RunSolverMode();
DOFS.Add("SplineLevelSet",pSAIDT.LevelSetOpti.GetLength());


Session ID: b2fb9694-235f-40e8-9667-5dba55eeec1d, DB path: 'SAIDT_db'
Session directory 'SAIDT_db\sessions\b2fb9694-235f-40e8-9667-5dba55eeec1d'.
Grid repartitioning method: METIS
Grid repartitioning options: 
Number of cell Weights: 0


*****************************************************************************************
*  X   X   DDD   GGG   -   III  SSS  TTTTT  *
*   X X    D  D G      -    I  S      T     *
*    X     D  D G  GG  -    I   SSS   T     *      XDG Implicit Shock Tracking Solver    
*   X X    D  D G   G  -    I      S  T     *
*  X   X   DDD   GGG   -   III  SSS   T     *
*****************************************************************************************
*  
*  solver:                             SAIDT.SAIDTMain
*  project:                            SAIDT_SplineLevelSet
*  session:                            SAIDT_SplineLevelSet
*  optimization problem:               FullEnRes
*  merit function:                     L1Merit
*  level set type:               

In [8]:
pSAIDT.ResNorms.SaveToTextFile(pSAIDT.Control.ProjectName+"_ResPlot.txt");
pSAIDT.obj_f_vals.SaveToTextFile(pSAIDT.Control.ProjectName+"_EnResPlot.txt");
pSAIDT.GetResPlot().ToGnuplot().PlotSVG(xRes:600,yRes:500)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -12 
 
 
 
 
 10 -10 
 
 
 
 
 10 -8 
 
 
 
 
 10 -6 
 
 
 
 
 10 -4 
 
 
 
 
 10 -2 
 
 
 
 
 10 0 
 
 
 
 
 10 2 
 
 
 
 
 0 
 
 
 
 
 5 
 
 
 
 
 10 
 
 
 
 
 15 
 
 
 
 
 20 
 
 
 
 
 25 
 
 
 
 
 30 
 
 
 
 
 35 
 
 
 
 
 40 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Iteration 
 
 
 
 
 ||R(z)|| 
 
 
 ||R(z)|| 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M388.5,57.1 L441.9,57.1 M53.9,84.5 L66.3,88.5 L78.7,98.5 L91.1,99.9 L103.5,101.4 L115.9,111.7
 L128.3,120.0 L140.7,135.6 L153.1,138.1 L165.5,141.4 L177.9,144.4 L190.4,148.5 L202.8,153.7 L215.2,160.4
 L227.6,168.5 L240.0,178.0 L252.4,187.3 L264.8,200.1 L277.2,214.3 L289.6,231.9 L302.0,254.6 L314.4,285.7
 L326.8,313.4 L339.2,326.3 L351.6,345.9 L364.0,358.8 L376.4,359.7 L388.8,366.8 L401.2,366.4 L413.6,368.1
 L426.0,369.0 L438.5,371.3 L450.9,371.2 L463.3,371.2 L475.7,371.2 L488.1,371.2 L500.5,371.2 L512.9,371.2
 L525.3,371.2 L537.7,371.2 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ||r(z)|| 
 
 
 ||r(z)|| 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M388.5,81.1 L441.9,81.1 M53.9,98.9 L66.3,95.4 L78.7,108.6 L91.1,109.8 L103.5,115.7 L115.9,127.0
 L128.3,133.0 L140.7,159.0 L153.1,171.5 L165.5,189.4 L177.9,227.1 L190.4,201.7 L202.8,181.2 L215.2,206.7
 L227.6,191.0 L240.0,187.8 L252.4,220.7 L264.8,216.9 L277.2,229.9 L289.6,246.7 L302.0,268.0 L314.4,311.4
 L326.8,325.8 L339.2,330.0 L351.6,354.6 L364.0,370.2 L376.4,372.1 L388.8,387.8 L401.2,387.4 L413.6,385.1
 L426.0,388.3 L438.5,390.0 L450.9,394.3 L463.3,394.3 L475.7,394.3 L488.1,394.3 L500.5,394.3 L512.9,394.3
 L525.3,394.3 L537.7,394.3 '/>

In [9]:
var si= database.Sessions.Pick(0);
pSAIDT.PlotShadowFields(si);

## SpecFemField

In [14]:
using SAIDT;
BoSSS.Solution.Application.InitMPI();
var pSAIDT = new SAIDTMain();
var CSAIDT = SAIDTHardCodedControl.StraightShock(
                    dbPath: dbPath,
                    MaxIterations: 100,
                    dgDegree: 0,
                    numOfCellsX: 5,
                    numOfCellsY: 5,
                    OptiNumOfCellsX: 5,
                    OptiNumOfCellsY: 5,
                    agg: 0.4,
                    ImmediatePlotPeriod: -1,
                    optiLevelSetType: OptiLevelSetType.SpecFemField,
                    LSDegree: 3,
                    isFarConfig:false
                    );
                    CSAIDT.minimalSQPIterations=new int[] {100,100,100,100};
CSAIDT.LevelSetDegree = CSAIDT.OptiLevelSetDegree*2;
CSAIDT.SessionName="SAIDT_CGLevelSet";
CSAIDT.ProjectName="SAIDT_CGLevelSet";
pSAIDT.Init(CSAIDT);
pSAIDT.RunSolverMode();
DOFS.Add("SpecFemField",pSAIDT.LevelSetOpti.GetLength());

Session ID: 16965f68-8133-4dd2-a9b5-814fac88b887, DB path: 'SAIDT_db'
Session directory 'SAIDT_db\sessions\16965f68-8133-4dd2-a9b5-814fac88b887'.
Grid repartitioning method: METIS
Grid repartitioning options: 
Number of cell Weights: 0


*****************************************************************************************
*  X   X   DDD   GGG   -   III  SSS  TTTTT  *
*   X X    D  D G      -    I  S      T     *
*    X     D  D G  GG  -    I   SSS   T     *      XDG Implicit Shock Tracking Solver    
*   X X    D  D G   G  -    I      S  T     *
*  X   X   DDD   GGG   -   III  SSS   T     *
*****************************************************************************************
*  
*  solver:                             SAIDT.SAIDTMain
*  project:                            SAIDT_CGLevelSet
*  session:                            SAIDT_CGLevelSet
*  optimization problem:               FullEnRes
*  merit function:                     L1Merit
*  level set type:                     Sp

Solve_Direct fail in fallback seq (#0): Linear Solver: High residual from direct solver ilPSP.LinSolvers.PARDISO.PARDISOSolver.
    L2 Norm of RHS:         0.182387253740761
    L2 Norm of Solution:    0.0238293585692048
    L2 Norm of Residual:    0.028923529257619766
    Relative Residual norm: 3.942233265842278E-05
    Matrix Inf norm:        733.6838615875286



alpha, M, predM  1, 0.003736687870757112, 0.0007096028781444295
alpha, M, predM  0.5, 0.001141974923760716, 0.0007096303267511918
alpha, M, predM  0.25, 0.0006996314056118274, 0.000709644051054573
l2: ||R0||= 1.3989501E-02, f_err=2.983543E-02, f_phi=0E+00, Alpha = 2.5E-01, Gamma = 2.96E-01, Kappa = 1E+03, Mu = 8.49E-03, QuadOrder=7
**NOT** terminating now, minimal number of iterations not reached


Starting Opt-Iter No.13: 
alpha, M, predM  1, 0.0037415084413612654, 0.000603566250284058
alpha, M, predM  0.5, 0.0010843778802151443, 0.0006036003949565605
alpha, M, predM  0.25, 0.0005002696291486358, 0.0006036174672928118
l2: ||R0||= 1.2360968E-02, f_err=2.683991E-02, f_phi=0E+00, Alpha = 2.5E-01, Gamma = 1.98E-01, Kappa = 1E+03, Mu = 5.29E-03, QuadOrder=7
**NOT** terminating now, minimal number of iterations not reached


Starting Opt-Iter No.14: 
alpha, M, predM  1, 0.00038499615806642537, 0.00039369286145052643
l2: ||R0||= 1.311634E-02, f_err=2.637946E-02, f_phi=0E+00, Alpha = 1E+00, G

Solve_Direct fail in fallback seq (#0): Linear Solver: High residual from direct solver ilPSP.LinSolvers.PARDISO.PARDISOSolver.
    L2 Norm of RHS:         0.12756585682231344
    L2 Norm of Solution:    0.1735995123979139
    L2 Norm of Residual:    0.0005922354680440049
    Relative Residual norm: 8.040261572874151E-07
    Matrix Inf norm:        736.5873145745165



alpha, M, predM  1, 0.000905570215061373, 0.0001678420875787709
alpha, M, predM  0.5, 0.00038414954865712513, 0.0001678588173091804
alpha, M, predM  0.25, 0.0002286179395399439, 0.00016786718217438518
alpha, M, predM  0.125, 0.00017920309834360118, 0.00016787136460698755
alpha, M, predM  0.0625, 0.00016835229052319825, 0.00016787345582328872
alpha, M, predM  0.03125, 0.0001647647753609765, 0.00016787450143143932
l2: ||R0||= 8.9900838E-03, f_err=1.813204E-02, f_phi=0E+00, Alpha = 3.13E-02, Gamma = 1E-04, Kappa = 1E+03, Mu = 2.03E-05, QuadOrder=7
**NOT** terminating now, minimal number of iterations not reached


Starting Opt-Iter No.36: 
alpha, M, predM  1, 0.0010906474019724915, 0.00016453369492544227
alpha, M, predM  0.5, 0.0002739035495815652, 0.0001645501235584243
alpha, M, predM  0.25, 0.00018898985271213365, 0.00016455833787491533
alpha, M, predM  0.125, 0.00017250174394646748, 0.00016456244503316086
alpha, M, predM  0.0625, 0.00016572291929335355, 0.00016456449861228361
alpha, M,

...

Solve_Direct fail in fallback seq (#0): Linear Solver: High residual from direct solver ilPSP.LinSolvers.PARDISO.PARDISOSolver.
    L2 Norm of RHS:         0.10067096602371108
    L2 Norm of Solution:    4.242463543183944
    L2 Norm of Residual:    34.434518126198576
    Relative Residual norm: 0.041827863634817754
    Matrix Inf norm:        823.243530361304



alpha, M, predM  1, 0.0013310662911583224, 0.00010609137206556847
alpha, M, predM  0.5, 0.00042086349631459576, 0.00010610193931696968
alpha, M, predM  0.25, 0.0002115733001477551, 0.00010610722294267028
alpha, M, predM  0.125, 0.00014450985771013127, 0.00010610986475552059
alpha, M, predM  0.0625, 0.0001177623077307108, 0.00010611118566194575
alpha, M, predM  0.03125, 0.00011042950844796779, 0.00010611184611515833
alpha, M, predM  0.015625, 0.00010709720820147088, 0.00010611217634176461
alpha, M, predM  0.0078125, 0.00010568001130241298, 0.00010611234145506776
l2: ||R0||= 7.2035098E-03, f_err=1.451855E-02, f_phi=0E+00, Alpha = 7.81E-03, Gamma = 1E-04, Kappa = 1E+03, Mu = 1.95E-05, QuadOrder=7
**NOT** terminating now, minimal number of iterations not reached


Starting Opt-Iter No.92: 


Solve_Direct fail in fallback seq (#0): Linear Solver: High residual from direct solver ilPSP.LinSolvers.PARDISO.PARDISOSolver.
    L2 Norm of RHS:         0.11963887218657504
    L2 Norm of Solution:    0.013541533949936777
    L2 Norm of Residual:    0.004470763131767663
    Relative Residual norm: 5.423726684769164E-06
    Matrix Inf norm:        824.2972759527871



alpha, M, predM  1, 0.002132938626181901, 0.00010547100425806526
alpha, M, predM  0.5, 0.0006614796935995494, 0.00010548153856632144
alpha, M, predM  0.25, 0.0002696010480376962, 0.00010548680572044953
alpha, M, predM  0.125, 0.00015618960731184493, 0.00010548943929751359
alpha, M, predM  0.0625, 0.0001259083962191227, 0.0001054907560860456
alpha, M, predM  0.03125, 0.00011246272472557824, 0.00010549141448031161
alpha, M, predM  0.015625, 0.00010650268598753452, 0.00010549174367744463
alpha, M, predM  0.0078125, 0.00010482335024868178, 0.00010549190827601112
l2: ||R0||= 7.179573E-03, f_err=1.447244E-02, f_phi=0E+00, Alpha = 7.81E-03, Gamma = 1E-04, Kappa = 1E+03, Mu = 6.68E-06, QuadOrder=7
**NOT** terminating now, minimal number of iterations not reached


Starting Opt-Iter No.93: 
alpha, M, predM  1, 0.0013237363711008886, 0.0001049877579604738
alpha, M, predM  0.5, 0.0004181295110071654, 0.0001049982153738005
alpha, M, predM  0.25, 0.00021013663344836514, 0.00010500344408046386
alpha

Error: System.ArgumentException: An item with the same key has already been added. Key: SpecFemField
   at System.Collections.Generic.Dictionary`2.TryInsert(TKey key, TValue value, InsertionBehavior behavior)
   at System.Collections.Generic.Dictionary`2.Add(TKey key, TValue value)
   at Submission#15.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [15]:
pSAIDT.ResNorms.SaveToTextFile(pSAIDT.Control.ProjectName+"_ResPlot.txt");
pSAIDT.obj_f_vals.SaveToTextFile(pSAIDT.Control.ProjectName+"_EnResPlot.txt");
pSAIDT.GetResPlot().ToGnuplot().PlotSVG(xRes:600,yRes:500)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -3 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 10 0 
 
 
 
 
 10 1 
 
 
 
 
 0 
 
 
 
 
 20 
 
 
 
 
 40 
 
 
 
 
 60 
 
 
 
 
 80 
 
 
 
 
 100 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Iteration 
 
 
 
 
 ||R(z)|| 
 
 
 ||R(z)|| 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M388.5,57.1 L441.9,57.1 M45.6,103.2 L50.6,116.2 L55.7,159.3 L60.7,169.2 L65.8,187.9 L70.8,269.1
 L75.9,275.6 L80.9,277.5 L86.0,275.8 L91.0,281.5 L96.1,289.5 L101.1,292.4 L106.1,292.6 L111.2,297.3
 L116.2,298.0 L121.3,305.2 L126.3,306.9 L131.4,307.8 L136.4,308.7 L141.5,309.4 L146.5,310.1 L151.5,310.2
 L156.6,311.3 L161.6,312.7 L166.7,312.7 L171.7,312.7 L176.8,312.7 L181.8,312.9 L186.9,312.9 L191.9,313.1
 L197.0,313.5 L202.0,313.6 L207.0,313.7 L212.1,313.7 L217.1,314.2 L222.2,314.6 L227.2,314.8 L232.3,315.3
 L237.3,315.8 L242.4,316.0 L247.4,316.4 L252.4,316.8 L257.5,317.4 L262.5,318.1 L267.6,318.1 L272.6,318.3
 L277.7,318.4 L282.7,318.4 L287.8,318.5 L292.8,318.5 L297.9,318.6 L302.9,318.6 L307.9,318.8 L313.0,318.8
 L318.0,319.5 L323.1,319.8 L328.1,319.8 L333.2,319.8 L338.2,320.0 L343.3,320.0 L348.3,320.1 L353.3,320.2
 L358.4,320.5 L363.4,320.9 L368.5,321.1 L373.5,321.4 L378.6,322.3 L383.6,322.4 L388.7,322.5 L393.7,322.6
 L398.8,322.6 L403.8,322.7 L408.8,322.7 L413.9,322.7 L418.9,322.8 L424.0,322.9 L429.0,323.1 L434.1,323.1
 L439.1,323.1 L444.2,323.3 L449.2,323.4 L454.2,323.5 L459.3,323.5 L464.3,323.6 L469.4,323.8 L474.4,323.9
 L479.5,324.0 L484.5,324.1 L489.6,324.1 L494.6,324.3 L499.7,324.3 L504.7,324.4 L509.7,324.5 L514.8,324.6
 L519.8,324.7 L524.9,324.7 L529.9,324.7 L535.0,325.0 L540.0,325.0 L545.1,325.0 L550.1,325.0 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ||r(z)|| 
 
 
 ||r(z)|| 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M388.5,81.1 L441.9,81.1 M45.6,151.9 L50.6,146.0 L55.7,204.2 L60.7,208.9 L65.8,222.8 L70.8,297.1
 L75.9,306.2 L80.9,310.9 L86.0,313.5 L91.0,316.4 L96.1,323.8 L101.1,325.1 L106.1,326.0 L111.2,331.5
 L116.2,328.9 L121.3,336.7 L126.3,338.4 L131.4,338.9 L136.4,339.8 L141.5,340.5 L146.5,341.4 L151.5,341.5
 L156.6,342.3 L161.6,343.6 L166.7,343.7 L171.7,343.7 L176.8,343.7 L181.8,343.8 L186.9,343.9 L191.9,344.1
 L197.0,344.4 L202.0,344.5 L207.0,344.6 L212.1,344.7 L217.1,345.1 L222.2,345.5 L227.2,345.8 L232.3,346.3
 L237.3,346.9 L242.4,347.0 L247.4,347.5 L252.4,347.8 L257.5,348.4 L262.5,349.1 L267.6,349.1 L272.6,349.3
 L277.7,349.3 L282.7,349.4 L287.8,349.5 L292.8,349.5 L297.9,349.6 L302.9,349.6 L307.9,349.8 L313.0,349.7
 L318.0,350.5 L323.1,350.7 L328.1,350.7 L333.2,350.8 L338.2,351.0 L343.3,351.0 L348.3,351.0 L353.3,351.0
 L358.4,351.4 L363.4,351.9 L368.5,352.1 L373.5,352.3 L378.6,353.3 L383.6,353.3 L388.7,353.4 L393.7,353.5
 L398.8,353.5 L403.8,353.6 L408.8,353.6 L413.9,353.7 L418.9,353.7 L424.0,353.8 L429.0,354.0 L434.1,354.0
 L439.1,354.0 L444.2,354.2 L449.2,354.3 L454.2,354.4 L459.3,354.4 L464.3,354.5 L469.4,354.7 L474.4,354.9
 L479.5,354.9 L484.5,355.0 L489.6,355.0 L494.6,355.2 L499.7,355.2 L504.7,355.3 L509.7,355.4 L514.8,355.5
 L519.8,355.6 L524.9,355.6 L529.9,355.6 L535.0,355.9 L540.0,355.9 L545.1,355.9 L550.1,356.0 '/>

In [16]:
var si= database.Sessions.Pick(0);
pSAIDT.PlotShadowFields(si);

In [17]:
using System.IO;
static void AppendRowToCSV(string filePath, double[] rowData)
    {
        // Create a StreamWriter in append mode to append rows to the CSV file
        using (StreamWriter writer = new StreamWriter(filePath, true))
        {
            int cols = rowData.Length;

            // Write double values of the new row to the file
            for (int i = 0; i < cols; i++)
            {
                writer.Write(rowData[i]);

                // Add a comma after each value except for the last one in a row
                if (i < cols - 1)
                {
                    writer.Write(";");
                }
            }

            // Move to the next line after writing the row
            writer.WriteLine();
        }
    }
    static void WriteCSV(string filePath, string[] data)
    {
        // Create a StreamWriter to write to the CSV file
        using (StreamWriter writer = new StreamWriter(filePath))
        {
            int cols = data.Length;

            // Loop through the data array and write to the file
                for (int j = 0; j < cols; j++)
                {
                    // Write each cell value to the file
                    writer.Write(data[ j]);

                    // Add a comma after each cell value except for the last one in a row
                    if (j < cols - 1)
                    {
                        writer.Write(";");
                    }
                }

                // Move to the next line after each row
                writer.WriteLine();
            
        }
    }
    static void DeleteCSVFiles()
    {
        try
        {
            // Get all CSV files in the specified directory
            string[] csvFiles = Directory.GetFiles("*.csv");

            // Delete each CSV file
            foreach (string file in csvFiles)
            {
                File.Delete(file);
                Console.WriteLine($"Deleted: {file}");
            }
        }
        catch (Exception ex)
        {
            Console.WriteLine($"Error deleting CSV files: {ex.Message}");
        }
    }

## Measure wall-clock time for computation of sensitivities and residuals 

In [18]:
using System.Diagnostics;
BoSSS.Solution.Application.InitMPI();
DeleteCSVFiles();

int nRefs=4;
var optLsTypes=new OptiLevelSetType[] {OptiLevelSetType.SinglePhaseField,OptiLevelSetType.SplineLevelSet,OptiLevelSetType.SpecFemField};
var CellsVsRuntime_dRdPhi = MultidimensionalArray.Create(nRefs,optLsTypes.Length +1);
var CellsVsRuntime_Residuals = MultidimensionalArray.Create(nRefs,optLsTypes.Length +1);
var CellsVsRuntime_dRdU = MultidimensionalArray.Create(nRefs,optLsTypes.Length +1);
string[] stringArray = new string[] { "nCells" }.Concat(optLsTypes.Select(x => x.ToString())).ToArray();

//Create files for results
string filePathdRdPhi = "TimeDRdPhi.csv";
string filePathdRdU = "TimedRdU.csv";
string filePathResiduals = "TimeResiduals.csv";
string filePathDOFs = "DOFs.csv";
WriteCSV(filePathdRdPhi,stringArray);
WriteCSV(filePathdRdU,stringArray);
WriteCSV(filePathResiduals,stringArray);
WriteCSV(filePathDOFs,stringArray);

for(int n =0; n< nRefs;n++){
    //counter for level set type
    int iLS=1;
    //save cell resolution
    int reso= (int) Math.Pow(2,n+1);
    CellsVsRuntime_dRdPhi[n,0]=reso*reso;
    double[] dRdPhiResults= new double[optLsTypes.Length +1];
    dRdPhiResults[0]=reso*reso;
    double[] dRdUResults=dRdPhiResults.CloneAs();
    double[] resiudalResults=dRdPhiResults.CloneAs();
    double[] DOFs=dRdPhiResults.CloneAs();
    foreach(OptiLevelSetType lstype in optLsTypes){
        var pSAIDT = new SAIDTMain();
        var CSAIDT = SAIDTHardCodedControl.StraightShock(
                            dbPath: null,
                            MaxIterations: 1,
                            dgDegree: 0,
                            numOfCellsX: reso,
                            numOfCellsY: reso,
                            OptiNumOfCellsX: reso,
                            OptiNumOfCellsY: reso,
                            agg: 0.4,
                            ImmediatePlotPeriod: -1,
                            optiLevelSetType: lstype,
                            LSDegree: 3,
                            isFarConfig:true);

        pSAIDT.Init(CSAIDT);    
        pSAIDT.InitializeEverything();
        //pSAIDT.RunSolverMode();

        //Measure time for creation of Jacobians dRdPhi
        Stopwatch stopwatch = new Stopwatch();stopwatch.Start();
        pSAIDT.FD_LevelSet();
        stopwatch.Stop();
        CellsVsRuntime_dRdPhi[n,iLS] =stopwatch.ElapsedMilliseconds;
        dRdPhiResults[iLS]=stopwatch.ElapsedMilliseconds;

        //Measure time for creation of Jacobians dRdU
        stopwatch = new Stopwatch();stopwatch.Start();
        pSAIDT.Oproblem.GetJacobians(pSAIDT.ConservativeFields, pSAIDT.XSpatialOperator.LinearizationHint);
        stopwatch.Stop(); 
        CellsVsRuntime_dRdU[n,iLS] =stopwatch.ElapsedMilliseconds;
        dRdUResults[iLS]=stopwatch.ElapsedMilliseconds;

        //Measure time for evaluation of the Residuals
        stopwatch = new Stopwatch();stopwatch.Start();
        pSAIDT.ComputeResiduals();
        stopwatch.Stop(); Console.WriteLine("Elapsed Time: " + stopwatch.ElapsedMilliseconds + "ms");
        CellsVsRuntime_Residuals[n,iLS] =stopwatch.ElapsedMilliseconds;
        resiudalResults[iLS]=stopwatch.ElapsedMilliseconds;

        DOFs[iLS]=pSAIDT.LevelSetOpti.GetLength();

        iLS++;
    }
    // Append the new row to the existing CSV file
    AppendRowToCSV(filePathdRdPhi, dRdPhiResults);
    AppendRowToCSV(filePathdRdU, dRdUResults);
    AppendRowToCSV(filePathResiduals, resiudalResults);
    AppendRowToCSV(filePathDOFs, DOFs);
}

Error deleting CSV files: The filename, directory name, or volume label syntax is incorrect. : 'c:\experimental\public\examples\ShockFitting\Studies\ComparisonLevelSets\*.csv'
Session ID: 00000000-0000-0000-0000-000000000000, DB path: 'EMPTY'
Session directory 'EMPTY\sessions\00000000-0000-0000-0000-000000000000'.
Grid repartitioning method: METIS
Grid repartitioning options: 
Number of cell Weights: 0


*****************************************************************************************
*  X   X   DDD   GGG   -   III  SSS  TTTTT  *
*   X X    D  D G      -    I  S      T     *
*    X     D  D G  GG  -    I   SSS   T     *      XDG Implicit Shock Tracking Solver    
*   X X    D  D G   G  -    I      S  T     *
*  X   X   DDD   GGG   -   III  SSS   T     *
*****************************************************************************************
*  
*  solver:                             SAIDT.SAIDTMain
*  project:                            ScalarAdvection
*  session:             

In [19]:
public PlotFormat GetFormat(int count){
    
    var allPT = new BoSSS.Solution.Gnuplot.PointTypes[] { PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle,PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle};
    var allC = new BoSSS.Solution.Gnuplot.LineColors[] { LineColors.Blue, LineColors.Black, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red};
    var Fmt = new PlotFormat();
    Fmt.PointSize = 0.8;
    Fmt.LineWidth = 1;    
    Fmt.Style     = Styles.LinesPoints;

    Fmt.LineColor = allC[count];
    Fmt.PointType =  allPT[count];
    return Fmt;
}

In [20]:
CellsVsRuntime_dRdPhi

Dimension,2
Lengths,"[ 4, 4 ]"
Storage,"[ 4, 1072, 166, 966, 16, 5891, 415, 4927, 64, 32444, 1074, 36302, 256, 269255, 4169, 328148 ]"
IsContinuous,True
StructureType,General
Length,16
NoOfCols,4
NoOfRows,4
IsLocked,False


## Plot time needed for computation of dRdPhi against number of cells

In [21]:
var plot = new Plot2Ddata();
int count=0;
for(int iLS=0;iLS<CellsVsRuntime_dRdPhi.Lengths[1]-1; iLS ++){
    plot.AddDataGroup(optLsTypes[iLS].ToString(),CellsVsRuntime_dRdPhi.ExtractSubArrayShallow(-1,0).To1DArray(),CellsVsRuntime_dRdPhi.ExtractSubArrayShallow(-1,iLS+1).To1DArray(), GetFormat(count));
    count++;
}
plot.Xlabel="number of Cells";
plot.LogY = true;
plot.LogX = true;
plot.ShowXtics = true;
plot.ToGnuplot().PlotSVG(xRes:1200,yRes:600)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 2 
 
 
 
 
 10 3 
 
 
 
 
 10 4 
 
 
 
 
 10 5 
 
 
 
 
 10 6 
 
 
 
 
 10 0 
 
 
 
 
 10 1 
 
 
 
 
 10 2 
 
 
 
 
 10 3 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 number of Cells 
 
 
 
 
 SinglePhaseField 
 
 
 SinglePhaseField 
 
 
 
 
 
 
 
 
 
 
 SplineLevelSet 
 
 
 SplineLevelSet 
 
 
 
 
 
 
 
 
 
 
 SpecFemField 
 
 
 SpecFemField